In [1]:
from dotenv import load_dotenv

In [2]:
load_dotenv()
import os

In [3]:
from supabase import create_client, Client
supabase_url = os.environ.get("SUPABASE_URL")
supabase_api_key = os.environ.get("SUPBASE_KEY")

supabase: Client = create_client(supabase_url, supabase_api_key)

In [5]:
from openai import OpenAI
client = OpenAI(api_key = os.getenv("OPENAI_API_KEY"))

In [6]:
from datetime import datetime

In [17]:
def funding_research(date):
    result = supabase.table('curation_selected_items')\
        .select('event_id, output,sources, news_date','topic')\
    .eq('news_date', date)\
    .eq('topic', 'Funding')\
        .order('created_at', desc=False)\
        .execute()
    
    return result.data

In [21]:
event_to_research = funding_research('2026-02-21')

In [33]:
def openai_research_funding(research_input):
    for i in research_input:
        response = client.responses.create(
            model = "gpt-5-nano",
            tools = [{
                "type": "web_search"
            }],
            include = ["web_search_call.action.sources"],
            input = f"""
            You are a funding intelligence research analyst for Krux.
        
              TASK:
              Create a fact-checked funding/M&A brief for ONE AI news event.
            
              PRIMARY GOAL:
              Extract only decision-useful, verifiable financing facts for working professionals and investors.
            
              OUTPUT FORMAT:
              - 8 to 10 bullet points max, if you're using more you're being redundant.
              - Each bullet must end with inline citations in this format: [Source: <publisher>, <url>]
              - No prose before/after bullets.
            
              MANDATORY ORDER:
              1) WHAT IS NEW (first bullet, one sentence): exact event today (round/M&A/strategic investment/debt/fund close).
              2) TRANSACTION FACTS:
                 - deal type (Seed/Series A/B/.../M&A/secondary/debt/fund close)
                 - announced amount and currency
                 - post-money/pre-money valuation (if disclosed)
                 - lead investor(s) and key participants
              3) STRUCTURE DETAILS:
                 - equity vs debt vs secondary (if available)
                 - tranche/timing/conditionality (if available)
              4) USE OF PROCEEDS:
                 - what capital will be used for (compute, hiring, expansion, product, etc.)
              5) STRATEGIC SIGNAL:
                 - concrete strategic angle only if explicitly reported (distribution, infra lock-in, GTM partnership, geopolitics, regulation)
              6) STATUS / CERTAINTY:
                 - confirmed vs reported/rumored; include company response (confirmed/declined comment/no comment)
            
              STRICT RULES:
              - Funding-first scope: do NOT drift into generic product/policy narrative unless directly tied to this transaction.
              - No historical funding rounds unless directly required to understand this specific transaction.
              - No opinions, forecasts, or speculation.
              - If multiple sources conflict, include both numbers/claims and label clearly as conflicting.
              - Prefer primary/high-signal sources (company release, filings, Reuters/FT/WSJ/Bloomberg/TechCrunch).
              - Ignore low-signal aggregator claims unless corroborated by a stronger source.
            
              QUALITY CHECK BEFORE FINALIZING:
              - Every bullet contains at least one citation.
              - No repeated facts.
              - First bullet is strictly the new event.
              - Amount, investor, valuation, and status are explicitly covered (or marked not disclosed).
            
              Event to research:
              Event: {i['output']}
              Sources: {i['sources']}
            """,
        )

        output = response.output_text
        print(output)

        final_dictionary = {
            'event_id': i['event_id'],
            'news_date': i['news_date'],
            'output': output,
            'model_provider': 'openai',
            'topic': i['topic']
        }
        print(final_dictionary)

        save_research(final_dictionary)

        print("✅ Done")

In [35]:
event_to_research[1:]

[{'event_id': '1319_2026-02-21',
  'output': 'Code Metal closed a $125 million Series B (reported) led by Salesforce Ventures to modernize legacy and defense software with AI-driven tooling, marking a significant funding round in the AI-for-legacy-systems space.',
  'sources': [{'url': 'https://mlq.ai/news/code-metal-lands-125m-series-b-to-modernize-legacy-defense-software-with-ai/',
    'name': 'MLQ.ai'}],
  'news_date': '2026-02-21',
  'topic': 'Funding'},
 {'event_id': '1301_2026-02-21',
  'output': 'Peak XV (formerly Sequoia India & SEA leadership) announced a $1.3 billion close across new India/Asia-focused funds to invest in startups including AI, fintech and consumer sectors. The raise represents a major deployment of capital targeting AI and regional startup expansion and positions Peak XV as a larger regional investor with significant dry powder.',
  'sources': [{'url': 'https://techcrunch.com/2026/02/20/peak-xv-raises-1-3b-doubles-down-on-ai-as-global-vc-rivalry-in-india-heat

In [36]:
openai_research_funding(event_to_research[1:])

- Code Metal closed a $125 million Series B led by Salesforce Ventures to modernize legacy defense software with AI-driven tooling. [Source: MLQ.ai, https://mlq.ai/news/code-metal-lands-125m-series-b-to-modernize-legacy-defense-software-with-ai/][Source: Salesforce Ventures, https://salesforceventures.com/perspectives/verified-code-for-mission-critical-systems/]

- Transaction facts: Series B funding; announced amount $125 million (USD); post-money valuation $1.25 billion; lead investor Salesforce Ventures; other participants include Accel, B Capital, Shield Capital, Smith Point Capital, RTX Ventures, Overmatch, and additional backers. [Source: MLQ.ai, https://mlq.ai/news/code-metal-lands-125m-series-b-to-modernize-legacy-defense-software-with-ai/][Source: eWeek, https://www.eweek.com/news/code-metal-valuation-ai-code-translation/]

- Structure details: equity financing (Series B); no debt or secondary component disclosed; timing reported as February 2026; no tranche or conditionality 

In [31]:
def save_research(research_json):
    supabase.table('research_assistant').insert({
        'event_id': research_json['event_id'],
        'model_provider': research_json['model_provider'],
        'news_date': research_json['news_date'],
        'output': research_json['output'],
        'topic': research_json['topic']
    }).execute()